In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler, FunctionTransformer
import category_encoders as ce

In [3]:
import dagshub
dagshub.init(repo_owner='sourav664', repo_name='real-estate-hybrid-app', mlflow=True)


Accessing as sourav664

Initialized MLflow to track repo "sourav664/real-estate-hybrid-app"

Repository sourav664/real-estate-hybrid-app initialized!

In [4]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow")

In [5]:
# mlflow experiment

mlflow.set_experiment("Exp 1 - Model Selection")

<Experiment: artifact_location='mlflow-artifacts:/e3f1b77bb6984c30bf68e5e81ceb1bb7', creation_time=1764310688561, experiment_id='0', last_update_time=1764310688561, lifecycle_stage='active', name='Exp 1 - Model Selection', tags={}>

In [6]:
from sklearn import set_config

set_config(transform_output="pandas")

## Load the data

In [7]:
from pathlib import Path

path = Path.cwd().parent
data_path = path / 'data/raw'

In [8]:
df = pd.read_csv(f"{data_path}/final_data.csv")

In [9]:
X = df.drop(columns=["price"])
y = df["price"]

In [10]:
df.duplicated().sum()

np.int64(5808)

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
     

In [13]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (38187, 11)
The shape of test data is (9547, 11)


In [14]:
ohe_encode = ['transactiontype','region','propertytype','furnished','ageofcons']
target_encode = ['locality']
robust_scaler = ['bedrooms','bathrooms','balconies','superbuiltupareasqft']

In [15]:
pt = FunctionTransformer(np.log1p, inverse_func=np.expm1, validate=True)
y_train_pt = pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt = pt.transform(y_test.values.reshape(-1,1))

c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))
c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))
c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))


In [16]:
preprocessor = ColumnTransformer(
    [
        ('ohe_encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ohe_encode),
        ('robust_scaler', RobustScaler(), robust_scaler)
    ],remainder="passthrough", n_jobs=-1, force_int_remainder_cols=False,verbose_feature_names_out=False
)


preprocessor.set_output(transform="pandas")

,transformers,"[('ohe_encode', ...), ('robust_scaler', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,False
,categories,'auto'
,drop,'first'
,sparse_output,False


In [17]:
# Creating a pipeline
pipeline = Pipeline([
    ('target_encoder', ce.TargetEncoder(cols=target_encode)),
    ('preprocessor', preprocessor)
    
])

In [18]:
# do data preprocessing

X_train_trans = pipeline.fit_transform(X_train, y_train)

X_test_trans = pipeline.transform(X_test)

c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


In [19]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import optuna
from sklearn.metrics import r2_score, mean_absolute_error

c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
def objective(trial):
    with mlflow.start_run(nested=True):
        model_name = trial.suggest_categorical("model",["RF","KNN","GB","XGB","LGBM"])
                
        if model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,200)
            max_depth_rf = trial.suggest_int("max_depth_rf",2,20)
            model = RandomForestRegressor(n_estimators=n_estimators_rf,
                                        max_depth=max_depth_rf,
                                        random_state=42,
                                        n_jobs=-1)

        elif model_name == "GB":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,200)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",0,1)
            max_depth_gb = trial.suggest_int("max_depth_gb",2,20)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb,
                                                learning_rate=learning_rate_gb,
                                                max_depth=max_depth_gb,
                                                random_state=42)

        elif model_name == "KNN":
            n_neighbors_knn = trial.suggest_int("n_neighbors_knn",1,25)
            weights_knn = trial.suggest_categorical("weights_knn",["uniform","distance"])
            model = KNeighborsRegressor(n_neighbors=n_neighbors_knn,
                                        weights=weights_knn,n_jobs=-1)

        elif model_name == "XGB":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,200)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",0.1,0.5)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",2,20)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                    learning_rate=learning_rate_xgb,
                                    max_depth=max_depth_xgb,
                                    random_state=42,
                                    n_jobs=-1)

        elif model_name == "LGBM":
            n_estimators_lgbm = trial.suggest_int("n_estimators_lgbm",10,200)
            learning_rate_lgbm = trial.suggest_float("learning_rate_lgbm",0.1,0.5)
            max_depth_lgbm = trial.suggest_int("max_depth_lgbm",2,20)
            model = LGBMRegressor(n_estimators=n_estimators_lgbm,
                                    learning_rate=learning_rate_lgbm,
                                    max_depth=max_depth_lgbm,
                                    random_state=42)
        # train the model
        model.fit(X_train_trans,y_train_pt)

        # log model params
        mlflow.log_params(model.get_params())

        # get the predictions
        y_pred_train = model.predict(X_train_trans)
        y_pred_test = model.predict(X_test_trans)

        # get the actual predictions values
        y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
        y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))

        # calculate the error
        error = mean_absolute_error(y_test,y_pred_test_org)
        
        # r2_score
        r2 = r2_score(y_test,y_pred_test_org)

        # log model_name
        mlflow.log_param("model",model_name)

        # log error
        mlflow.log_metric("MAE",error)

        # log r2_score
        mlflow.log_metric("r2",r2)

        return r2

In [21]:
# create optuna study
study = optuna.create_study(direction="maximize",study_name="model_selection")

with mlflow.start_run(run_name="Best Model") as parent:
    # optimize the objective function
    study.optimize(objective,n_trials=30,n_jobs=-1)

    # log the best parameters
    mlflow.log_params(study.best_params)

    # log the best score
    mlflow.log_metric("best_score",study.best_value)

[I 2025-11-30 17:39:12,827] A new study created in memory with name: model_selection
c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\sklearn\ensemble\_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\sklearn\ensemble\_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


🏃 View run receptive-worm-618 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/f59f99052b894700818cd6b9ed9f77f5
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0


[I 2025-11-30 17:39:25,074] Trial 3 finished with value: 0.7716103976700943 and parameters: {'model': 'RF', 'n_estimators_rf': 120, 'max_depth_rf': 6}. Best is trial 3 with value: 0.7716103976700943.
c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\lightgbm\basic.py:374: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")
c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\lightgbm\basic.py:374: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")


🏃 View run serious-moose-404 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/1602d70f616e47edb14638b50f0aa107
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0


[I 2025-11-30 17:39:53,489] Trial 8 finished with value: 0.8563636512784829 and parameters: {'model': 'RF', 'n_estimators_rf': 152, 'max_depth_rf': 11}. Best is trial 8 with value: 0.8563636512784829.
c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\sklearn\ensemble\_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


...

🏃 View run powerful-vole-861 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/91a51cbf28464ee2bc1a14fa09827824
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0
🏃 View run fearless-worm-839 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/4161c7e25e9142a4953f3f1a052f822d
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0


[I 2025-11-30 17:42:41,986] Trial 26 finished with value: 0.8330752795865191 and parameters: {'model': 'LGBM', 'n_estimators_lgbm': 22, 'learning_rate_lgbm': 0.48246118179864816, 'max_depth_lgbm': 4}. Best is trial 19 with value: 0.8812877995384193.
[I 2025-11-30 17:42:45,037] Trial 27 finished with value: 0.7804667889383071 and parameters: {'model': 'LGBM', 'n_estimators_lgbm': 24, 'learning_rate_lgbm': 0.4798337967830924, 'max_depth_lgbm': 2}. Best is trial 19 with value: 0.8812877995384193.


🏃 View run loud-cod-261 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/7f01f48992ad4c0db8f97ff64f98bb2d
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0
🏃 View run delicate-slug-353 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/4ddca7ffa8fa4ea097a30ecafc7aaf6e
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0


[I 2025-11-30 17:42:51,990] Trial 28 finished with value: 0.8626626526983985 and parameters: {'model': 'LGBM', 'n_estimators_lgbm': 25, 'learning_rate_lgbm': 0.3347702114597547, 'max_depth_lgbm': 10}. Best is trial 19 with value: 0.8812877995384193.
[I 2025-11-30 17:42:53,015] Trial 29 finished with value: 0.8836617607049171 and parameters: {'model': 'LGBM', 'n_estimators_lgbm': 199, 'learning_rate_lgbm': 0.3289192832305745, 'max_depth_lgbm': 10}. Best is trial 29 with value: 0.8836617607049171.


🏃 View run Best Model at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/d3a78c3330cd4a579e545ae270866414
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0


In [22]:
study.best_params

{'model': 'LGBM',
 'n_estimators_lgbm': 199,
 'learning_rate_lgbm': 0.3289192832305745,
 'max_depth_lgbm': 10}

In [23]:
# train the model on best parameters

lgbm = LGBMRegressor(**study.best_params)

lgbm.fit(X_train_trans,y_train_pt)

[LightGBM] [Warning] Unknown parameter: learning_rate_lgbm
[LightGBM] [Warning] Unknown parameter: model
[LightGBM] [Warning] Unknown parameter: n_estimators_lgbm
[LightGBM] [Warning] Unknown parameter: max_depth_lgbm


c:\Users\soura\anaconda3\envs\.venv\Lib\site-packages\lightgbm\basic.py:374: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: learning_rate_lgbm
[LightGBM] [Warning] Unknown parameter: model
[LightGBM] [Warning] Unknown parameter: n_estimators_lgbm
[LightGBM] [Warning] Unknown parameter: max_depth_lgbm
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.188311 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 542
[LightGBM] [Info] Number of data points in the train set: 38187, number of used features: 22
[LightGBM] [Info] Start training from score 1.335152


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [24]:
# get the predictions
y_pred_train = lgbm.predict(X_train_trans)
y_pred_test = lgbm.predict(X_test_trans)

[LightGBM] [Warning] Unknown parameter: learning_rate_lgbm
[LightGBM] [Warning] Unknown parameter: model
[LightGBM] [Warning] Unknown parameter: n_estimators_lgbm
[LightGBM] [Warning] Unknown parameter: max_depth_lgbm
[LightGBM] [Warning] Unknown parameter: learning_rate_lgbm
[LightGBM] [Warning] Unknown parameter: model
[LightGBM] [Warning] Unknown parameter: n_estimators_lgbm
[LightGBM] [Warning] Unknown parameter: max_depth_lgbm


In [25]:
# get the actual predictions values

y_pred_train_org = pt.inverse_transform(y_pred_train.reshape(-1,1))
y_pred_test_org = pt.inverse_transform(y_pred_test.reshape(-1,1))

In [26]:
print(f"The train error is {mean_absolute_error(y_train,y_pred_train_org):.2f} lakhs")
print(f"The test error is {mean_absolute_error(y_test,y_pred_test_org):.2f} lakhs")

The train error is 0.63 lakhs
The test error is 0.67 lakhs


In [27]:
print(f"The r2 score is {r2_score(y_train,y_pred_train_org):.2f}")
print(f"The r2 score is {r2_score(y_test,y_pred_test_org):.2f}")

The r2 score is 0.89
The r2 score is 0.87


In [28]:
# dataframe of results

study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_learning_rate_gb,params_learning_rate_lgbm,params_learning_rate_xgb,params_max_depth_gb,params_max_depth_lgbm,params_max_depth_rf,params_max_depth_xgb,params_model,params_n_estimators_gb,params_n_estimators_lgbm,params_n_estimators_rf,params_n_estimators_xgb,params_n_neighbors_knn,params_weights_knn,state
0,0,0.873431,2025-11-30 17:39:13.459072,2025-11-30 17:41:38.001834,0 days 00:02:24.542762,NaN,0.213729,NaN,NaN,14.0,NaN,NaN,LGBM,NaN,104.0,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.871329,2025-11-30 17:39:13.460920,2025-11-30 17:41:30.991789,0 days 00:02:17.530869,NaN,NaN,0.245471,NaN,NaN,NaN,7.0,XGB,NaN,NaN,NaN,165.0,NaN,NaN,COMPLETE
2,2,0.877156,2025-11-30 17:39:13.461864,2025-11-30 17:41:28.993023,0 days 00:02:15.531159,NaN,0.497410,NaN,NaN,5.0,NaN,NaN,LGBM,NaN,89.0,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.771610,2025-11-30 17:39:13.462868,2025-11-30 17:39:25.073954,0 days 00:00:11.611086,NaN,NaN,NaN,NaN,NaN,6.0,NaN,RF,NaN,NaN,120.0,NaN,NaN,NaN,COMPLETE
4,4,0.818777,2025-11-30 17:39:13.464204,2025-11-30 17:40:07.052111,0 days 00:00:53.587907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KNN,NaN,NaN,NaN,NaN,21.0,distance,COMPLETE
5,5,0.723549,2025-11-30 17:39:13.465417,2025-11-30 17:39:58.546817,0 days 00:00:45.081400,0.038228,NaN,NaN,11.0,NaN,NaN,NaN,GB,38.0,NaN,NaN,NaN,NaN,NaN,COMPLETE
6,6,0.861297,2025-11-30 17:39:13.466448,2025-11-30 17:41:58.005136,0 days 00:02:44.538688,0.346076,NaN,NaN,10.0,NaN,NaN,NaN,GB,172.0,NaN,NaN,NaN,NaN,NaN,COMPLETE
7,7,0.829137,2025-11-30 17:39:13.467789,2025-11-30 17:40:03.063795,0 days 00:00:49.596006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KNN,NaN,NaN,NaN,NaN,5.0,uniform,COMPLETE
8,8,0.856364,2025-11-30 17:39:25.084216,2025-11-30 17:39:53.489228,0 days 00:00:28.405012,NaN,NaN,NaN,NaN,NaN,11.0,NaN,RF,NaN,NaN,152.0,NaN,NaN,NaN,COMPLETE
9,9,0.851471,2025-11-30 17:39:53.496159,2025-11-30 17:40:21.276043,0 days 00:00:27.779884,0.152632,NaN,NaN,5.0,NaN,NaN,NaN,GB,48.0,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [29]:
study.trials_dataframe()['params_model'].value_counts()

params_model
LGBM    15
RF       6
KNN      4
GB       4
XGB      1
Name: count, dtype: int64

In [30]:
# avg scores for all tested models

study.trials_dataframe().groupby("params_model")['value'].mean().sort_values()

params_model
RF      0.765610
GB      0.816221
KNN     0.816838
LGBM    0.858640
XGB     0.871329
Name: value, dtype: float64